# Módulo 2 Implementación de un modelo de deep learning
## Docente: Benjamin
## José Antonio López Saldaña A01710367
---

*Evaluación de modelos de Deepfake 7-clases (CNN de 0 y CNN pre-creada)*

## Evaluación de modelo Deepfake 7-clases

## (Evaluación de CNN 128 x 128)
---
Pipeline de inferencia para videos

In [ ]:
# Si no los tienes, descomenta una vez:
# !mkdir -p /content/models
# !wget -q -O /content/models/deploy.prototxt https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
# !wget -q -O /content/models/res10_300x300_ssd_iter_140000.caffemodel https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

**Definiciones/Configuración del entorno de dataset**

*Defino la ruta y carpetas donde está la información del modelo creado*

In [ ]:
from pathlib import Path
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd

# Clases EXACTAS que usaste al entrenar
METHODS = [
    "original", "DeepFakeDetection", "Deepfakes",
    "Face2Face", "FaceShifter", "FaceSwap", "NeuralTextures"
]
IDX_ORIGINAL = METHODS.index("original")

IMG_SIZE = (128, 128)  # <-- tu modelo actual
TH_FRAME_REALFAKE = 0.5   # umbral p_fake por frame
TH_VIDEO_REALFAKE = 0.5   # umbral p_fake promedio por video


/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.5.3, so it will not be used.
  warnings.warn(


**Detector de rostros (OpenCV DNN)**

*Detectar y recortar una sola cara de una imagen usando el detector DNN de OpenCV, devolverla como un recorte cuadrado del tamaño (128×128)*

In [ ]:
DNN_PROTO = "/content/models/deploy.prototxt"
DNN_WEIGHTS = "/content/models/res10_300x300_ssd_iter_140000.caffemodel"

_dnn = cv2.dnn.readNetFromCaffe(DNN_PROTO, DNN_WEIGHTS)


def crop_one_face_bgr(img_bgr, out_size=IMG_SIZE[0], conf_thresh=0.5):
    """
    Detecta y recorta UNA cara del frame usando OpenCV DNN.
    Devuelve un recorte cuadrado BGR de tamaño out_size x out_size.
    """
    h, w = img_bgr.shape[:2]
    blob = cv2.dnn.blobFromImage(
        cv2.resize(img_bgr, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), False, False
    )
    _dnn.setInput(blob)
    detections = _dnn.forward()

    conf = 0.0
    if detections.shape[2] > 0:
        i = np.argmax(detections[0, 0, :, 2])
        conf = float(detections[0, 0, i, 2])

    if conf >= conf_thresh:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        x1, y1, x2, y2 = box.astype(int)
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        face = img_bgr[y1:y2, x1:x2]
        if face.size == 0:
            s = min(h, w)
            y = (h - s) // 2
            x = (w - s) // 2
            face = img_bgr[y:y+s, x:x+s]
    else:
        # Fallback: recorte centrado cuadrado
        s = min(h, w)
        y = (h - s) // 2
        x = (w - s) // 2
        face = img_bgr[y:y+s, x:x+s]

    return cv2.resize(face, (out_size, out_size))

In [ ]:
# -------------------------------
# 3) CARGAR MODELO ENTRENADO
# -------------------------------

CKPT_DIR = Path("/content/ffpp_checkpoints_efficient")

MODEL_CANDIDATES = [
    CKPT_DIR / "best_model_auc.keras",
    CKPT_DIR / "best_model_acc.keras",
]

def load_trained_model():
    """
    Carga el modelo guardado (.keras) de tu CNN eficiente 7 clases.
    Usa compile=False porque solo queremos hacer predicciones.
    """
    for p in MODEL_CANDIDATES:
        if p.exists():
            print(f"✓ Cargando modelo: {p}")
            return tf.keras.models.load_model(p, compile=False)
    raise FileNotFoundError(
        f"❌ No encuentro ningún modelo .keras en {CKPT_DIR}. "
        f"Esperaba: {[str(p) for p in MODEL_CANDIDATES]}"
    )

In [ ]:
# -------------------------------
# 4) PREPROCESAMIENTO PARA LA CNN
# -------------------------------

def preprocess_face_bgr(img_bgr):
    """
    Convierte BGR → RGB y prepara el tensor para el modelo.

    IMPORTANTE:
    - Tu modelo hace `Rescaling(1./255)` adentro.
    - Por eso aquí NO normalizamos, solo convertimos a float32 [0-255].
    """
    if img_bgr is None or img_bgr.size == 0:
        raise ValueError("face vacío")

    if img_bgr.shape[:2] != IMG_SIZE:
        img_bgr = cv2.resize(img_bgr, IMG_SIZE)

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    x = img_rgb.astype(np.float32)  # (128,128,3)
    return x

In [ ]:
# -------------------------------
# 5) PREDICCIÓN POR VIDEO
# -------------------------------

def predict_video(model, video_path: Path,
                  max_frames=4,
                  save_csv=None,
                  conf_thresh=0.5):
    """
    Extrae ~1 fps, detecta caras, predice y devuelve:
    - video_label_rf: "fake" o "real" (basado en p_fake promedio)
    - video_p_fake: probabilidad media de fake [0-1]
    - video_top_method: clase 7-way más probable a nivel video
    - video_top_score: probabilidad de esa clase
    - per_frame: lista de diccionarios con resultados por frame
    """
    per_frame = []
    count = 0

    try:
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            raise RuntimeError(f"No puedo abrir {video_path}")

        fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
        step = max(int(round(fps)), 1)  # ~1 frame por segundo
        i = 0

        print(f"\n📹 Procesando video: {video_path.name}")
        print(f"   FPS: {fps:.1f} | Step: {step} | Max frames: {max_frames}")
        print("   " + "="*50)

        while True:
            ok, fr = cap.read()
            if not ok:
                break

            if i % step == 0:
                try:
                    # Detectar y recortar cara a 128x128 (aprox.)
                    face128 = crop_one_face_bgr(fr, out_size=IMG_SIZE[0],
                                                conf_thresh=conf_thresh)

                    # Preprocesar
                    x = preprocess_face_bgr(face128)
                    x = np.expand_dims(x, 0)  # (1,128,128,3)

                    # Predicción 7 clases
                    probs = model.predict(x, verbose=0)[0]  # shape: (7,)
                    top_idx = int(np.argmax(probs))
                    top_method = METHODS[top_idx]
                    top_score = float(probs[top_idx])

                    # Probabilidad "real/fake" derivada
                    p_real = float(probs[IDX_ORIGINAL])
                    p_fake = float(1.0 - p_real)
                    label_rf = "fake" if p_fake >= TH_FRAME_REALFAKE else "real"

                    per_frame.append({
                        "frame_idx": i,
                        "label": label_rf,          # real/fake
                        "p_fake": p_fake,
                        "p_real": p_real,
                        "top_method": top_method,   # clase 7-way más probable
                        "top_score": top_score,
                        "probs": probs.tolist(),    # vector completo 7D
                        "err": ""
                    })

                    print(
                        f"   Frame {i:4d} → {label_rf.upper()} | "
                        f"p_fake={p_fake:.3f} | "
                        f"{top_method} ({top_score:.3f})"
                    )

                    count += 1
                    if max_frames and count >= max_frames:
                        break

                except Exception as e:
                    per_frame.append({
                        "frame_idx": i,
                        "label": "error",
                        "p_fake": np.nan,
                        "p_real": np.nan,
                        "top_method": "",
                        "top_score": np.nan,
                        "probs": [np.nan]*len(METHODS),
                        "err": repr(e)
                    })
                    print(f"   Frame {i:4d} → ERROR: {repr(e)}")

            i += 1

        cap.release()

    except Exception as e:
        per_frame.append({
            "frame_idx": -1,
            "label": "error",
            "p_fake": np.nan,
            "p_real": np.nan,
            "top_method": "",
            "top_score": np.nan,
            "probs": [np.nan]*len(METHODS),
            "err": f"video_open/iter: {repr(e)}"
        })
        print(f"❌ Error abriendo video: {repr(e)}")

    # -----------------------
    # Agregación por video
    # -----------------------
    probs_valid = [
        np.array(d["probs"], dtype=np.float32)
        for d in per_frame
        if d["label"] != "error"
    ]

    if len(probs_valid) > 0:
        mean_probs = np.mean(probs_valid, axis=0)  # (7,)
        video_top_idx = int(np.argmax(mean_probs))
        video_top_method = METHODS[video_top_idx]
        video_top_score = float(mean_probs[video_top_idx])

        video_p_real = float(mean_probs[IDX_ORIGINAL])
        video_p_fake = float(1.0 - video_p_real)
        video_label_rf = (
            "fake" if video_p_fake >= TH_VIDEO_REALFAKE else "real"
        )
    else:
        video_top_method = ""
        video_top_score = np.nan
        video_p_fake = np.nan
        video_label_rf = "error"

    # Guardar CSV opcional
    if save_csv:
        pd.DataFrame(per_frame).to_csv(save_csv, index=False)
        print(f"   💾 CSV guardado: {save_csv}")

    return (
        video_label_rf,  # "real"/"fake"/"error"
        video_p_fake,    # prob. media de fake
        video_top_method,
        video_top_score,
        per_frame,
    )

In [ ]:
# -------------------------------
# 6) GENERAR VIDEO ANOTADO
# -------------------------------

def annotate_and_save_sample(video_path: Path, per_frame, out_path: Path,
                             sample_rate=1):
    """
    Genera un video demo con las predicciones superpuestas.
    Usa:
    - label (real/fake)
    - p_fake
    - top_method (clase 7-way dominante)
    """
    picked = [
        d for i, d in enumerate(per_frame)
        if (i % sample_rate == 0) and d["label"] != "error"
    ]
    idx2rec = {d["frame_idx"]: d for d in picked}

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise RuntimeError(f"No puedo abrir {video_path}")

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    writer = cv2.VideoWriter(
        str(out_path),
        cv2.VideoWriter_fourcc(*"mp4v"),
        10,
        (w, h)
    )

    i = 0
    while True:
        ok, fr = cap.read()
        if not ok:
            break

        d = idx2rec.get(i)
        if d:
            text = (
                f"{d['label'].upper()} | "
                f"fake={d['p_fake']:.2f} | "
                f"{d['top_method']} ({d['top_score']:.2f})"
            )
            color = (0, 0, 255) if d["label"] == "fake" else (0, 255, 0)

            cv2.rectangle(fr, (10, 10), (800, 80), (0, 0, 0), -1)
            cv2.putText(
                fr, text, (20, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2, cv2.LINE_AA
            )

            writer.write(fr)
        i += 1

    cap.release()
    writer.release()
    print(f"   🎬 Video demo guardado: {out_path}")


In [ ]:
# -------------------------------
# 7) EJECUCIÓN DE EJEMPLO
# -------------------------------

if __name__ == "__main__":
    # Cargar modelo
    model = load_trained_model()
    print(f"Input shape del modelo: {model.input_shape}\n")

    # Video a analizar (CAMBIA ESTA RUTA)
    VIDEO_PATH = Path(
        "/content/ffpp_c23/Face2Face/003_000.mp4"
    )

    if not VIDEO_PATH.exists():
        print(f"❌ Video no encontrado: {VIDEO_PATH}")
        print("   Actualiza VIDEO_PATH con la ruta correcta")
    else:
        (
            video_label_rf,
            video_p_fake,
            video_top_method,
            video_top_score,
            per_frame,
        ) = predict_video(
            model,
            VIDEO_PATH,
            max_frames=8,  # puedes subir o bajar este número
            save_csv=Path("/content/predicciones_video_7clases.csv"),
            conf_thresh=0.5
        )

        # Resultado final
        print("\n" + "="*60)
        print("🎯 RESULTADO FINAL (VIDEO)")
        print("="*60)
        print(f"   Video:          {VIDEO_PATH.name}")
        print(f"   Real/Fake:      {video_label_rf.upper()} "
              f"(p_fake={video_p_fake:.3f})")
        print(f"   Clase 7-way:    {video_top_method} "
              f"({video_top_score:.3f})")
        print(f"   Frames OK:      "
              f"{len([d for d in per_frame if d['label'] != 'error'])}")
        print("="*60 + "\n")

        # Generar video demo (opcional)
        demo_path = Path("/content/demo_prediccion_7clases.mp4")
        annotate_and_save_sample(VIDEO_PATH, per_frame, demo_path, sample_rate=1)

✓ Cargando modelo: /content/ffpp_checkpoints_efficient/best_model_auc.keras
Input shape del modelo: (None, 128, 128, 3)


📹 Procesando video: 003_000.mp4
   FPS: 25.0 | Step: 25 | Max frames: 8
   Frame    0 → FAKE | p_fake=0.957 | Face2Face (0.906)
   Frame   25 → FAKE | p_fake=0.980 | Face2Face (0.930)
   Frame   50 → FAKE | p_fake=0.939 | Face2Face (0.758)
   Frame   75 → FAKE | p_fake=0.992 | Face2Face (0.969)
   Frame  100 → FAKE | p_fake=0.947 | Face2Face (0.888)
   Frame  125 → FAKE | p_fake=0.974 | Face2Face (0.948)
   Frame  150 → FAKE | p_fake=0.972 | Face2Face (0.935)
   Frame  175 → FAKE | p_fake=0.941 | Face2Face (0.710)
   💾 CSV guardado: /content/predicciones_video_7clases.csv

🎯 RESULTADO FINAL (VIDEO)
   Video:          003_000.mp4
   Real/Fake:      FAKE (p_fake=0.962)
   Clase 7-way:    Face2Face (0.880)
   Frames OK:      8

   🎬 Video demo guardado: /content/demo_prediccion_7clases.mp4


## Evaluación de modelo Deepfake 7-clases

## (Modelo preentrenado)
---
jhgshgjsdghksd